In [0]:
from pyspark.sql import SparkSession
!pip install yfinance
import yfinance as yf
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('Project').getOrCreate()
from pyspark.sql.functions import when
from pyspark.sql.functions import max
from pyspark.sql.functions import greatest,col,lit

In [0]:
Ticker_df = spark.read.csv('/FileStore/tables/constituents_csv.csv',inferSchema=True,header=True)

In [0]:
Ticker_df.show(5)

+------+-------------------+-----------+
Symbol| Name| Sector|
+------+-------------------+-----------+
 MMM| 3M Company|Industrials|
 AOS| A.O. Smith Corp|Industrials|
 ABT|Abbott Laboratories|Health Care|
 ABBV| AbbVie Inc.|Health Care|
 ABMD| ABIOMED Inc|Health Care|
+------+-------------------+-----------+
only showing top 5 rows

In [0]:
stocks = Ticker_df.select('Symbol').collect()

In [0]:
#yahoo finance recommendations by firms for msft
l=['Goldman Sachs','JP Morgan','Morgan Stanley','Barclays','UBS','Bank of America','Credit Suisse','Citigroup','Deutsche Bank','Wells Fargo']

msft = yf.Ticker("MSFT")
msft_sp_new = spark.createDataFrame(msft.recommendations.reset_index()).sort('Firm').dropDuplicates(['Firm'])
msft_sp_new=msft_sp_new.filter(F.col("Firm").isin(l)).sort('Firm').dropDuplicates(['Firm'])
msft_sp_new = msft_sp_new.withColumn("Action", when(msft_sp_new["Action"] == 'main', 1).when(msft_sp_new["Action"] == 'down', 2).when(msft_sp_new["Action"] == 'up', 3).when(msft_sp_new["Action"] == 'init', 4).when(msft_sp_new["Action"] == 'reit', 5).otherwise(None))
msft_sp_new=msft_sp_new.groupBy('To Grade').pivot('Firm').max('Action').drop('To Grade')
# msft_df=msft_sp_new.select([max('Bank of America'),max('Barclays'),max('Credit Suisse'),max('Goldman Sachs'),max('JP Morgan'),max('Morgan Stanley'),max('UBS'),max('Wells Fargo'),max('Deutsche Bank'),max('Citigroup')])
msft_df=msft_sp_new.select(*[max(col(c)) for c in msft_sp_new.columns])

# msft_df=msft_df.withColumn('Average',(col('max(Bank of America)')+col('max(Barclays)')+col('max(Credit Suisse)')+col('max(Goldman Sachs)')+col('max(JP Morgan)')+col('max(UBS)')+col('max(Citigroup)')+col('max(Deutsche Bank)')+col('max(Wells Fargo)'))/10)
msft_df=msft_df.withColumn('Average',(]))
msft_df=msft_df.withColumn('Firm',lit('msft')).select('Firm','Average')
msft_df.show()

+--------------------+-------------+--------------+------------------+------------------+------------------+--------------+-------------------+--------+----------------+
max(Bank of America)|max(Barclays)|max(Citigroup)|max(Credit Suisse)|max(Deutsche Bank)|max(Goldman Sachs)|max(JP Morgan)|max(Morgan Stanley)|max(UBS)|max(Wells Fargo)|
+--------------------+-------------+--------------+------------------+------------------+------------------+--------------+-------------------+--------+----------------+
 2| 1| 2| 1| 1| 1| 1| 1| 1| 3|
+--------------------+-------------+--------------+------------------+------------------+------------------+--------------+-------------------+--------+----------------+

+----+-------+
Firm|Average|
+----+-------+
msft| 1.3|
+----+-------+

In [0]:
#similarly we collected data for all the tickers
for i,ticker in enumerate(stocks[0:1]):
  tickers = yf.Ticker(ticker.Symbol)
  try:
    msft_sp_new = spark.createDataFrame(tickers.recommendations.reset_index()).sort('Firm').dropDuplicates(['Firm'])
    msft_sp_new=msft_sp_new.filter(F.col("Firm").isin(l)).sort('Firm').dropDuplicates(['Firm'])
    msft_sp_new = msft_sp_new.withColumn("Action", when(msft_sp_new["Action"] == 'main', 1).when(msft_sp_new["Action"] == 'down', 2).when(msft_sp_new["Action"] == 'up', 3).when(msft_sp_new["Action"] == 'init', 4).when(msft_sp_new["Action"] == 'reit', 5).otherwise(None))
    msft_sp_new=msft_sp_new.groupBy('To Grade').pivot('Firm').max('Action').drop('To Grade')
    msft_sp_new.show()
    msft_df=msft_sp_new.select(*[max(col(c)) for c in msft_sp_new.columns])
    msft_df=msft_df.withColumn('Average',(col('max(Bank of America)')+col('max(Barclays)')+col('max(Credit Suisse)')+col('max(Goldman Sachs)')+col('max(JP Morgan)')+col('max(UBS)')+col('max(Citigroup)')+col('max(Deutsche Bank)')+col('max(Wells Fargo)'))/10)
    msft_df=msft_df.withColumn('Firm',lit('msft')).select('Firm','Average')
    all_df = msft_df.union(msft_df)
  except:
    print(tickers)

In [0]:
# to csv
all_df.repartition(1).write.csv('/FileStore/shared_uploads/saisuraja@usf.edu/Reco_csv_2',header=True)